# Percu 1

In [1]:
%matplotlib inline

import math
import numpy as np
import matplotlib.pyplot as plt
import scipy

plt.rcParams['text.usetex'] = True

import IPython
import soundfile as sf


In [2]:
def play(*args, **kwargs):
    IPython.display.display(IPython.display.Audio(*args, **kwargs))

## Poutre d'Euler Bernoulli

modèle simple et classique

### Hypotheses

- Solide élancé, $L \gg h, b$
- La forme de la section ne change pas au cours du temps -> pas de deformation
- Pas de cisaillement ($\Phi''' = 0$?) (cisaillement considéré par Timoshenko)
- Petites perturbations d'un état d'équilibre
- Materiau isotrope
- Que de la flexion
- Section constante

### ED

$$
EI \frac{\partial^4 y}{\partial x^4} + \rho S \frac{\partial^2 y}{\partial t^2} = 0
$$

$$\displaystyle c^2 = \frac{EI}{\rho S}$$

$$
\frac{\partial^4 y}{\partial x^4} + \frac{1}{c^2} \frac{\partial^2 y}{\partial t^2} = 0
$$


### Solutions

On cherche des solutions stationnaires de la forme

$$
y(x, t) = \phi(x) e^{j \omega t + \varphi_0}
$$

$$
\phi'''' e^{j \omega t} -  \frac{\omega^2}{c^2} \phi e^{j \omega t} = 0
$$

$$
\displaystyle \gamma(\omega)^4 =  \frac{\omega^2}{c^2}
$$

Donc

$$
\phi''''(x) = \gamma(\omega)^4 \phi(x)
$$

Solutions:

$$
\phi(x) =
A e^{j \gamma  x} +
B e^{-j \gamma  x} +
C e^{\gamma  x} +
D e^{-\gamma  x}
$$

ou encore

$$
\phi(x) =
a \cos \gamma x +
b \sin \gamma x +
c \cosh \gamma x +
d \sinh \gamma x
$$

## Conditions aux limites (Libre-Libre d'abords)

déplacement libre: $y = ???$

vitesse libre:  $y' = ???$

pas de forces (et de moments)

-> pas de force de cisaillement -> $y''' = 0$

-> pas de moment flechissant -> $y'' = 0$

$$
\phi''(x) = - a \gamma^2 cos( \gamma x)
- b \gamma^2 \sin( \gamma x)
+ c \gamma^2 \cosh( \gamma x)
+ d \gamma^2 \sinh( \gamma x)
$$

### Solutions:

$$
\cos \gamma L \cosh \gamma L = 1
$$

On trouve les $\gamma$ qui verifient cette equation, puis $\omega = c  \gamma^2$


## Application numerique

[SOPRANO GLOCKENSPIEL BAR A (1ST OCTAVE)](https://musicandschools.com/en/orff/887-soprano-glockenspiel-bar-a-1st-octave-8435040736336.html)

> Soprano glockenspiel bar A (1st octave). Bar made of steel tuned to A=440Hz. Grey finish. Musical notation in the middle of the bar. Approximate dimensions: 20 x 3 x 95 mm. 

$$L = 95 \, \textrm{mm}$$
$$b = 20 \, \textrm{mm}$$
$$h = 3 \, \textrm{mm}$$
$$E(\textrm{acier}) = 200 \, \textrm{GPa}$$
$$\rho(\textrm{acier}) = 7800 \, \textrm{kg}/\textrm{m}^3$$

In [3]:
# tout en SI

L = 94.1928005 / 1000 # ajuste a la main pour produire un La
b = 20.0 / 1000
h = 3.0 / 1000
E = 200_000_000_000
rho = 7800

S = b * h

# moment quadratique
I = b * h * h * h / 12

c = math.sqrt(E * I / (rho * S))

# releves graphiquement sur desmos
solutions = [
    4.73004,
    7.8532,
    10.99561,
    14.137164,
    17.27876,
    20.52035,
]

gammas = [solution / L for solution in solutions]

pulsations = [c * gamma * gamma for gamma in gammas]

frequences = [omega / (2 * math.pi) for omega in pulsations]

frequences[0] / 440

4.000000001126775

In [4]:
# Synthese

sampleRate = 44_100
duration = 5 # secs
N = int(duration * sampleRate)

buffer = np.zeros(N)
t = np.linspace(0, duration, N)

for f in frequences:
    y = np.cos(2 * np.pi * f * t)
    buffer += y

decay = 0.5 # secs
envelope = np.exp(-t / decay)

buffer = np.multiply(buffer, envelope)

def dB(db):
    return 10**(db / 20)


# normalize
amplitude = max(np.abs(buffer))
buffer = buffer / amplitude

# set volume
volume = -12
buffer = buffer * dB(volume)

maxAmplitude = max(np.abs(buffer))
dB_fullScale = 20 * np.log10(maxAmplitude)
print(f"Warning: the following sound has a volume of {dB_fullScale} dB FS")
print("If it reads a positive value, it WILL clip and destroy your ears")
print("play at your own risk")

play(buffer, rate=sampleRate)
sf.write('ding.wav', buffer, sampleRate)

If it reads a positive value, it WILL clip and destroy your ears
play at your own risk



## Annexe

### Dérivées de $y$

##### 1) $\displaystyle \frac{\partial y}{\partial x}$

Si $\theta$ est l'angle formé par la poutre, 
$\displaystyle \theta(x, t) \sim \tan(\theta(x, t)) = \frac{\partial y}{\partial x}(x, t)$

$\displaystyle \frac{\partial y}{\partial x}$ représente la pente/rotation de la poutre

##### 2) $\displaystyle \frac{\partial^2 y}{\partial x^2}$

Lié au rayon de courbure
$$
\frac{1}{\mathcal{R}} = 
\frac{\partial^2 y}{\partial x^2}
$$ 

Lié au moment fléchissant
$$
\mathcal{M} = EI \frac{\partial \theta}{\partial x}
$$
$$
\mathcal{M} = EI 
\frac{\partial^2 y}{\partial x^2}
$$

##### 3) $\displaystyle \frac{\partial^3 y}{\partial x^3}$

La où la courbure change
=> lié à la force de cisaillement
$$
\mathcal{T} = \frac{\partial}{\partial x}(EI
\frac{\partial^2 y}{\partial x^2}
)
$$

Et si $EI$ constant:
$$
\mathcal{T} = EI 
\frac{\partial^3 y}{\partial x^3}
$$